## Notebook to keep track of tabular data
PPMI [documentation](https://www.ppmi-info.org/sites/default/files/docs/PPMI%20Data%20User%20Guide%20v.1.0.pdf).

### Currently collating and tracking following assessments:
    - UPDRS
    - MoCA

### Notes
    - MDS-UPDRS_Part_I.csv has `NP1RTOT` i.e MDS-UPDRS Part I (Rater Completed) Total Score
    - MDS-UPDRS_Part_I_Patient_Questionnaire.csv has `NP1PTOT` i.e. MDS-UPDRS Part I (Patient Questionnaire) Total Score
    - UPDRS-3 has medication on/off info which needs to be filtered carefully. See PPMI documentation. 

### TODO
    - Add "INFODT" (i.e. date of assessment)
    - Add PAG specific columns (i.e. medication on/off scores)


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product

In [ ]:
ppmi_dir = "/home/nikhil/projects/Parkinsons/ppmi/releases/Dec_2022/"
tabular_dir = f"{ppmi_dir}tabular"
demographics_dir = f"{tabular_dir}/demographics"
study_data_dir = f"{tabular_dir}/assessments"

# Populate bids_id in the bagel.csv
bids_dir = f"{ppmi_dir}/bids"
bids_participants_tsv = f"{bids_dir}/participants.tsv"

# Available clinical visits (These are differently named than imaging sessions (i.e. session.csv))
# These are generated based on Vincent's notebooks/subject_profiling.ipynb
visits_csv = f"{demographics_dir}/event_session_visits.csv" 

clinical_assessment_files = {
                "updrs_1": "MDS-UPDRS_Part_I.csv",
                "updrs_1_PQ": "MDS-UPDRS_Part_I_Patient_Questionnaire.csv",
                "updrs_2_PQ": "MDS_UPDRS_Part_II__Patient_Questionnaire.csv",
                "updrs_3": "MDS_UPDRS_Part_III.csv",
                "updrs_4_MC":"MDS-UPDRS_Part_IV__Motor_Complications.csv",
                "moca":"Montreal_Cognitive_Assessment__MoCA_.csv"
}

bagel_csv = f"{tabular_dir}/bagel.csv"

### PPMI mr_proc manifest

In [ ]:
current_mr_proc_manifest_csv = f"{demographics_dir}/mr_proc_manifest.csv"
current_mr_proc_manifest_df = pd.read_csv(current_mr_proc_manifest_csv)

n_participants = len(current_mr_proc_manifest_df["participant_id"].unique())
print(f"n_participants: {n_participants}")

dx_groups = current_mr_proc_manifest_df["group"].unique()
print(f"groups: {dx_groups}")
select_dx_groups = ['PD', 'Prodromal', 'Control', 'SWEDD']
print(f"select groups: {select_dx_groups}")
current_mr_proc_manifest_df = current_mr_proc_manifest_df[current_mr_proc_manifest_df["group"].isin(select_dx_groups)]

n_participants = len(current_mr_proc_manifest_df["participant_id"].unique())
print(f"n_participants: {n_participants}")

# Add PATNO column name to match with PPMI study data
current_mr_proc_manifest_df["participant_id"] = current_mr_proc_manifest_df["participant_id"].astype(str)
current_mr_proc_manifest_df["PATNO"] = current_mr_proc_manifest_df["participant_id"]

current_mr_proc_manifest_df.head()

### Add available bids ids from participants.tsv

In [ ]:
bids_participants_df = pd.read_csv(bids_participants_tsv,sep="\t")
n_bids_participants = len(bids_participants_df["participant_id"].unique())
print(f"Available bids participants: {n_bids_participants}")
bids_participants_df["bids_id"] = bids_participants_df["participant_id"]
bids_participants_df["PATNO"] = bids_participants_df["participant_id"].str.split("-",expand=True)[1].astype(str)
current_mr_proc_manifest_df = pd.merge(current_mr_proc_manifest_df, bids_participants_df[["PATNO","bids_id"]], on="PATNO", how="left")

n_participants = len(current_mr_proc_manifest_df["participant_id"].unique())
print(f"n_participants: {n_participants}")

current_mr_proc_manifest_df.head()

### Available clinical visits (these are different than imaging sessions)
- Need to select a subset you expect data for 

### BIDS session code (infered from DICOMs) to LONI EVENT_IDs MAP
session_dict = {0: "Screening",1: "Baseline", 5: "Month 12",7: "Month 24",9: "Month 36",11: "Month 48"}

event_dict = {"BL": "Baseline", "SC":"Screening", "V04":"Month 12", "V06": "Month 24", "V08": "Month 36", "V10": "Month 48", "V12": "Month 60"}

In [ ]:
visit_df = pd.read_csv(visits_csv)
event_ids = visit_df["EVENT_ID"]
print(f"Number of total possible visits: {len(event_ids)}")
visit_df

### Read all UPDRS scores

#### UPDRS-1

In [ ]:
updrs_part_1_df = pd.read_csv(f"{study_data_dir}/{clinical_assessment_files['updrs_1']}")
n_records = len(updrs_part_1_df)
n_participants = len(updrs_part_1_df["PATNO"].unique())
n_events = len(updrs_part_1_df["EVENT_ID"].unique())
n_PAG = len(updrs_part_1_df["PAG_NAME"].unique())
print(f"n_records: {n_records}, n_participants={n_participants}, n_events={n_events}, n_PAG={n_PAG}")
updrs_part_1_df.head()

In [ ]:
updrs_part_1_PQ_df = pd.read_csv(f"{study_data_dir}/{clinical_assessment_files['updrs_1_PQ']}")
n_records = len(updrs_part_1_PQ_df)
n_participants = len(updrs_part_1_PQ_df["PATNO"].unique())
n_events = len(updrs_part_1_PQ_df["EVENT_ID"].unique())
n_PAG = len(updrs_part_1_PQ_df["PAG_NAME"].unique())
print(f"n_records: {n_records}, n_participants={n_participants}, n_events={n_events}, n_PAG={n_PAG}")
updrs_part_1_PQ_df.head()

In [ ]:
updrs_part_1_PQ_df.groupby(["PAG_NAME"]).count()

#### UPDRS-2

In [ ]:
updrs_part_2_PQ_df = pd.read_csv(f"{study_data_dir}/{clinical_assessment_files['updrs_2_PQ']}")
n_records = len(updrs_part_2_PQ_df)
n_participants = len(updrs_part_2_PQ_df["PATNO"].unique())
n_events = len(updrs_part_2_PQ_df["EVENT_ID"].unique())
n_PAG = len(updrs_part_2_PQ_df["PAG_NAME"].unique())
print(f"n_records: {n_records}, n_participants={n_participants}, n_events={n_events}, n_PAG={n_PAG}")
updrs_part_2_PQ_df.head()

In [ ]:
updrs_part_2_PQ_df.groupby(["PAG_NAME"]).count()

#### UPDRS-3
- this is most commonly used for motor function
- this is sensitive to medication "ON" and "OFF" stages

In [ ]:
updrs_part_3_df = pd.read_csv(f"{study_data_dir}/{clinical_assessment_files['updrs_3']}",low_memory=False)
n_records = len(updrs_part_3_df)
n_participants = len(updrs_part_3_df["PATNO"].unique())
n_events = len(updrs_part_3_df["EVENT_ID"].unique())
n_PAG = len(updrs_part_3_df["PAG_NAME"].unique())
print(f"n_records: {n_records}, n_participants={n_participants}, n_events={n_events}, n_PAG={n_PAG}")
updrs_part_3_df.head()

#### Plot `on` vs `off` scores

In [ ]:
plot_df = updrs_part_3_df.copy()
plot_df = plot_df[plot_df["EVENT_ID"].isin(visit_df["EVENT_ID"].values)]
participants_with_medication_info = plot_df[plot_df["PAG_NAME"]=="NUPDR3ON"]["PATNO"].values
print(f"participants_with_medication_info: {len(participants_with_medication_info)}")
plot_df = plot_df[plot_df["PATNO"].isin(participants_with_medication_info)]
sns.set(font_scale=1)
with sns.axes_style("whitegrid"):
    g = sns.catplot(x="EVENT_ID", y="NP3TOT",hue="PAG_NAME", data=plot_df, 
                    aspect=2, kind="box")

In [ ]:
updrs_part_3_df.groupby(["PAG_NAME"]).count()

#### UPDRS-4

In [ ]:
updrs_part_4_MC_df = pd.read_csv(f"{study_data_dir}/{clinical_assessment_files['updrs_4_MC']}")
n_records = len(updrs_part_4_MC_df)
n_participants = len(updrs_part_4_MC_df["PATNO"].unique())
n_events = len(updrs_part_4_MC_df["EVENT_ID"].unique())
n_PAG = len(updrs_part_4_MC_df["PAG_NAME"].unique())
print(f"n_records: {n_records}, n_participants={n_participants}, n_events={n_events}, n_PAG={n_PAG}")
updrs_part_4_MC_df.head()

### Read MoCA scores

In [ ]:
MoCA_df = pd.read_csv(f"{study_data_dir}/{clinical_assessment_files['moca']}")
n_records = len(MoCA_df)
n_participants = len(MoCA_df["PATNO"].unique())
n_events = len(MoCA_df["EVENT_ID"].unique())
n_PAG = len(MoCA_df["PAG_NAME"].unique())
print(f"n_records: {n_records}, n_participants={n_participants}, n_events={n_events}, n_PAG={n_PAG}")
MoCA_df.head()

### Merge demographics, all UPDRS and MoCA total Scores into single CSV

In [ ]:
participant_ids = current_mr_proc_manifest_df["participant_id"].values.astype(str)
print(f"n_participants = {len(participant_ids)}")
a,b = zip(*list(product(participant_ids,event_ids)))

bagel_df = pd.DataFrame()
bagel_df["participant_id"] = a
bagel_df["EVENT_ID"] = b

print(f"Creating empty dataframe with all ({len(bagel_df)}) possible records i.e. combinations of participant_id and EVENT_ID")
# merge manifest df (Scores will have multiple visits but manifest currently only has single visit)
# TODO 
# - initialize this with updated manifest
# - Add "INFODT" (i.e. date of assessment)
# - Add PAG specific columns (i.e. medication on/off scores)

baseline_visit = "BL"
current_mr_proc_manifest_df["EVENT_ID"] = baseline_visit
bagel_df = pd.merge(bagel_df, current_mr_proc_manifest_df, on=["participant_id","EVENT_ID"], how="left")
print(f"n_records all: {len(bagel_df)}")
# merge updrs and moca
total_col_dict = {
    "updrs_1": {"col":"NP1RTOT", "PAG":"NUPDRS1"}, 
    "updrs_1_PQ": {"col":"NP1PTOT", "PAG":"NUPDRS1P"},
    "updrs_2_PQ": {"col":"NP2PTOT", "PAG":"NUPDRS2P"},
    "updrs_3": {"col":"NP3TOT", "PAG":"NUPDRS3"},
    "updrs_4_MC":{"col":"NP4TOT", "PAG":"NUPDRS4"},
    "moca": {"col":"MCATOT", "PAG":"MOCA"}
}

assessment_cols = []
for k,v_dict in total_col_dict.items():
    score_df = pd.read_csv(f"{study_data_dir}/{clinical_assessment_files[k]}",low_memory=False)
    score_df["participant_id"] = score_df["PATNO"].astype(str)
    col = v_dict["col"]
    PAG = v_dict["PAG"] # medication stratification 
    print(f"Assessment + PAG: {v_dict}")
    score_df = score_df[score_df["PAG_NAME"]==PAG]
    bagel_df = pd.merge(bagel_df, score_df[["participant_id","EVENT_ID", col]], 
                        on=["participant_id","EVENT_ID"], how="left")

    # Assign availability status cols
    bagel_df[f"{col}_status"] = ~bagel_df[col].isna()
    assessment_cols.append(col)

    n_records = len(bagel_df)
    n_participants = len(bagel_df["PATNO"].unique())
    n_events = len(bagel_df["EVENT_ID"].unique())
    print(f"n_records: {n_records}, n_participants={n_participants}, n_events={n_events}")

bagel_df = pd.merge(bagel_df,visit_df, on=["EVENT_ID"], how="left")

print(f"n_records all: {len(bagel_df)}")
# Drop records i.e. (participant_id,EVENT_ID) with no available data (i.e. missing visits)
print("Dropping records i.e. (participant_id,EVENT_ID) with no available data")
check_data_cols = ["age","sex","group"] + assessment_cols
bagel_df = bagel_df[~bagel_df[check_data_cols].isna().all(axis=1)]
print(f"n_records (with available visits): {len(bagel_df)}")

bagel_df.head()

In [ ]:
bagel_df.groupby("EVENT_ID").count()

### Save bagel_df


In [ ]:
save_bagel = False

print(f"Save path: {bagel_csv}")
if save_bagel:
    bagel_df.to_csv(bagel_csv, index=None)
